## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [5]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

# Testing to make sure TensorFlow GPU is working
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4908570464288356736
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3805675520
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3574269181951922708
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


### Load data, shuffle data, normalize data, split data into training and validation

In [6]:
from sklearn.utils import shuffle

# Load the data
X_train = np.load("X_train_3.dat")
y_train = np.load("y_train_3.dat")


X_train, y_train = shuffle(X_train, y_train, random_state=42)
# reshape so in form for CNN-Keras
#X_train = X_train.reshape(X_train.shape[0], 174, 124, 1)

# Normalize the data
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

# split data into validation set
training_set_size = int(X_train.shape[0] * .80)
X_train_validation = X_train[training_set_size:, :,:,:]
y_train_validation = y_train[training_set_size:, :]
y_train = y_train[:training_set_size,:]
X_train = X_train[:training_set_size, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(9000, 124, 174, 3)
(9000, 10)
Validation x train set:(1800, 124, 174, 3)
X train set:(7200, 124, 174, 3)
Validation y train set:(1800, 10)
Y train set:(7200, 10)


## Build the CNN network

In [7]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (7,7), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(128, (11,11), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Conv2D(128, (7, 7), strides=(1,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(64, (7, 7), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

## Begin training the model

In [ ]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model_3splits_200epochs_midLR", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

adam = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(adam, 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=100 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 7200 samples, validate on 1800 samples
Epoch 1/100
7200/7200 [==============================] - 66s 9ms/step - loss: 2.0018 - acc: 0.3439 - val_loss: 1.4261 - val_acc: 0.4733
Epoch 2/100
7200/7200 [==============================] - 62s 9ms/step - loss: 1.5063 - acc: 0.4851 - val_loss: 1.7759 - val_acc: 0.4006
Epoch 3/100
7200/7200 [==============================] - 63s 9ms/step - loss: 1.2626 - acc: 0.5640 - val_loss: 1.1314 - val_acc: 0.6028
Epoch 4/100
7200/7200 [==============================] - 63s 9ms/step - loss: 1.0504 - acc: 0.6394 - val_loss: 1.3352 - val_acc: 0.5500
Epoch 5/100
7200/7200 [==============================] - 63s 9ms/step - loss: 0.8297 - acc: 0.7208 - val_loss: 1.1830 - val_acc: 0.5956
Epoch 6/100
7200/7200 [==============================] - 62s 9ms/step - loss: 0.6116 - acc: 0.8001 - val_loss: 1.1199 - val_acc: 0.6278
Epoch 7/100
7200/7200 [==============================] - 62s 9ms/step - loss: 0.5869 - acc: 0.8053 - val_loss: 0.8952 - val_acc: 0.6983


/home/anthony/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.335030). Check your callbacks.
  % delta_t_median)


7200/7200 [==============================] - 63s 9ms/step - loss: 0.1910 - acc: 0.9410 - val_loss: 0.8310 - val_acc: 0.7317
Epoch 11/100
7200/7200 [==============================] - 62s 9ms/step - loss: 0.1344 - acc: 0.9635 - val_loss: 0.8967 - val_acc: 0.7294
Epoch 12/100
7200/7200 [==============================] - 62s 9ms/step - loss: 0.1004 - acc: 0.9726 - val_loss: 0.8594 - val_acc: 0.7522
Epoch 13/100
7200/7200 [==============================] - 63s 9ms/step - loss: 0.0820 - acc: 0.9801 - val_loss: 0.9819 - val_acc: 0.7167
Epoch 14/100
7200/7200 [==============================] - 62s 9ms/step - loss: 0.0683 - acc: 0.9815 - val_loss: 0.8502 - val_acc: 0.7472
Epoch 15/100
7200/7200 [==============================] - 63s 9ms/step - loss: 0.0620 - acc: 0.9850 - val_loss: 0.8754 - val_acc: 0.7478
Epoch 16/100
7200/7200 [==============================] - 62s 9ms/step - loss: 0.0600 - acc: 0.9843 - val_loss: 0.8799 - val_acc: 0.7439
Epoch 17/100
7200/7200 [==============================

In [5]:
model.save("models/trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]


### Reset Keras

In [53]:
from keras import backend as K
K.clear_session()